In [1]:
import pandas as pd
import numpy as np
import pickle

In [3]:
#Atoms(symbols,positions)

In [7]:
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')
# structures = pd.read_csv('../Data/structures.csv')
# train_bonds = pd.read_csv('../Data/train_bonds.csv')
# test_bonds = pd.read_csv('../Data/test_bonds.csv')

In [2]:
train_stack = pd.read_csv('../Data/train_oof_0815.csv')

In [3]:
test_stack = pd.read_csv('../Data/test_oof_0815.csv')

In [8]:
features = ['gnn_0731', 'gnn_0802',
           'edge_0802', 'lgb_0729', 'lgb_0730', 'lgb_0803', 'edge_0804',
           'edge_0806', 'edge_0729', 'edge_0730', 'edge_0807',
           'edge_0808', 'edge_0729_100epoch', 'edge_0730_100epoch', 'edge_0811',
           'meta_0812', 'lgb_restacking_0812', 'edge_5fold_0729_160epoch',
           'edge_5fold_0730_160epoch', 'model3_0813', 'model3_0814','model3_0815']

In [10]:
stacking = pd.concat([train_stack.loc[:,['id']+features],test_stack.loc[:,['id']+features]],0)

In [12]:
del train_stack,test_stack

### coupling information

In [13]:
test['scalar_coupling_constant'] = np.nan

In [14]:
coupling = train.append(test, ignore_index=True)

In [17]:
coupling.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [22]:
assert np.all(coupling.id.values == stacking.id.values)

In [32]:
coupling = pd.merge(coupling,stacking,'left',on='id')

In [33]:
del stacking

In [36]:
coupling.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,gnn_0731,gnn_0802,edge_0802,lgb_0729,...,edge_0729_100epoch,edge_0730_100epoch,edge_0811,meta_0812,lgb_restacking_0812,edge_5fold_0729_160epoch,edge_5fold_0730_160epoch,model3_0813,model3_0814,model3_0815
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,85.204422,84.400238,85.675613,88.422145,...,87.437523,82.608986,84.599823,84.306496,85.499368,86.013916,81.654236,84.062859,84.311279,87.292900
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,-13.812207,-11.299141,-11.053956,-9.749846,...,-10.060014,-10.340228,-11.453266,-10.702221,-10.883259,-10.683383,-10.668419,-9.973827,-10.237222,-10.855716
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,-13.811005,-11.297125,-11.050260,-9.749846,...,-10.056600,-10.336302,-11.450315,-10.698669,-10.883259,-10.678482,-10.664890,-9.969814,-10.232733,-10.851676
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,-13.811016,-11.297044,-11.049506,-9.717249,...,-10.055735,-10.335059,-11.449178,-10.697636,-10.883259,-10.677493,-10.663880,-9.968416,-10.231138,-10.850039
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,85.205597,84.400703,85.687164,88.407545,...,87.447685,82.625580,84.607445,84.313400,85.512350,86.021858,81.653030,84.076500,84.317688,87.308350


In [37]:
coupling[['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN']]=pd.get_dummies(coupling.type)

In [39]:
coupling = coupling.groupby(['molecule_name'])

In [40]:
coupling_edge_index = {}
coupling_edge_attr = {}
coupling_edge_dist = {}
coupling_y = {}
coupling_id = {}
for k,v in coupling:
    coupling_edge_index[k] = v[['atom_index_0','atom_index_1']].values.T
    coupling_edge_attr[k] = v[['1JHC','1JHN','2JHC','2JHH','2JHN','3JHC','3JHH','3JHN']].values.astype(np.float32)
    coupling_edge_dist[k] = v[features].values.astype(np.float32)
    coupling_id[k] = v.id.values
    if not np.any(np.isnan(v.scalar_coupling_constant.values)):
        coupling_y[k] = v.scalar_coupling_constant.values.astype(np.float32)

In [41]:
with open('../Data/coupling_edge_index_stacking.pickle', 'wb') as handle:
    pickle.dump(coupling_edge_index, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/coupling_edge_attr_stacking.pickle', 'wb') as handle:
    pickle.dump(coupling_edge_attr, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/coupling_edge_feature_stacking.pickle', 'wb') as handle:
    pickle.dump(coupling_edge_dist, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/coupling_y_stacking.pickle', 'wb') as handle:
    pickle.dump(coupling_y, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/coupling_id_stacking.pickle', 'wb') as handle:
    pickle.dump(coupling_id, handle, protocol=pickle.HIGHEST_PROTOCOL)  